In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation,Bidirectional,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
import util
import hyperparameter_tuning
embeddings = None
from hyperopt import hp,STATUS_OK

In [35]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [36]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [37]:
processed, X_processed = prepare_for_cnn(train_df)

Percentage of words covered in the embeddings = 0.4937444933920705


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

In [14]:
def RNN():
    model = Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)])
    return model

In [15]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, 64)                71808     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 76,033
Trainable params: 76,033
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(X_train, y_train, epochs=50, shuffle=False, validation_split=0.0)
util.plot_history(history)

Epoch 1/50
160/160 [==============================] - 1s 8ms/step - loss: 0.6719 - accuracy: 0.5700
Epoch 2/50
160/160 [==============================] - 1s 8ms/step - loss: 0.6430 - accuracy: 0.5765
Epoch 3/50
160/160 [==============================] - 1s 8ms/step - loss: 0.6297 - accuracy: 0.5888
Epoch 4/50
160/160 [==============================] - 1s 8ms/step - loss: 0.6203 - accuracy: 0.6071
Epoch 5/50
160/160 [==============================] - 1s 7ms/step - loss: 0.6080 - accuracy: 0.6200
Epoch 6/50
160/160 [==============================] - 1s 8ms/step - loss: 0.5988 - accuracy: 0.6443
Epoch 7/50
160/160 [==============================] - 2s 11ms/step - loss: 0.5850 - accuracy: 0.6578
Epoch 8/50
160/160 [==============================] - 2s 11ms/step - loss: 0.5718 - accuracy: 0.6808
Epoch 9/50
160/160 [==============================] - 1s 9ms/step - loss: 0.5598 - accuracy: 0.6896
Epoch 10/50
160/160 [==============================] - 1s 8ms/step - loss: 0.5469 - accuracy: 0.70

KeyError: 'val_accuracy'

In [17]:
prediction = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [18]:
accuracy_score(y_test, prediction)

0.7934739355352168

In [19]:
f1_score(y_test, prediction)

0.7098937954164338

Hyperopt (bayesian)

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [13]:
processed = feature_builder.process_dataset(train_df)
X_processed = processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1])
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

Percentage of words covered in the embeddings = 0.4937444933920705


In [9]:
def gru(params):
    model = Sequential()
    model.add(Bidirectional(GRU(params['units']),input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(params['units2'], activation='relu'))
    model.add(Dropout(params['dropout']))
    model.add(Dense(1))
    
    
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
    
    result = model.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2)
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_accuracy']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}


In [10]:
space = {'units': hp.choice('units',[16,32,64]),
        'units2': hp.choice('units2',[16,32,64]),
        'dropout': hp.choice('dropout',[0.1,0.2,0.5,0.7])}

In [11]:
hyperparameter_tuning.bayesian_optimization(gru,space,max_evals=3)

Epoch 1/50                                           
  1/128 [..............................]             
 - ETA: 0s - loss: 0.7076 - accuracy: 0.4688         
                                                    
  5/128 [>.............................]             
 - ETA: 1s - loss: 0.7898 - accuracy: 0.4062         
                                                    
  7/128 [>.............................]             
 - ETA: 1s - loss: 0.7829 - accuracy: 0.4241         
                                                    
 11/128 [=>............................]             
 - ETA: 1s - loss: 0.7869 - accuracy: 0.4261         
                                                    
 15/128 [==>...........................]             
 - ETA: 1s - loss: 0.7768 - accuracy: 0.4458         
                                                    
 20/128 [===>..........................]             
 - ETA: 1s - loss: 0.7681 - accuracy: 0.4391         
                                 